## The Problem 

### Seating Arrangement in Covid

- Owners of a movie theater must ensure that they can safely seat customers at minimum 2 meters of distance, while trying to seat as many people as possible to maximize their profit. Customers often visit a stadium show in groups. Members of a group must be seated together, but spectators in different groups must be seated at a safe distance away from other groups. 
- We will approach this issue as a bin packing problem. Each group is treated as a ‘box’ with a buffer zone of one seat surrounding the group on all sides. Each of these‘augmented groups’ (groups with buffer space) can now be packed adjacent to each other to maximize thegroups seated in a stadium. 
- We define an augmented row to be 3 ‘theater rows’ (the row in which the group is seated plus the buffer row above and below that row). 
- The total number of augmented rows in the stadium to be (the total number of regular rows/3). 
- Groups can only be seated in a single row (not in multiple rows), so the maximum allowed size of an augmented group is less than or equal to  W, the number of seats in one row. 
- The demand (people/groups who want to be seated is known ahead of time). Out of these, some will be seated, and some will not be seated.

#### Intro to our Model:
- Cineplex is considering seating groups, indexed i \in I in an theatre with rows j \in J and the width of the rows to contain W seats.
- Each group was a width in number of seats, w. 
- Each row can contain up to W seats
- The objective is to minimize the total number of surplus(empty) seats in the theater to ensure that the maximium number of people can be seated. Social distancing is maintained due to the augumented group structure, where there will be a minimum of 2 seats between 2 groups seated in the same row.




## Construction Heuristic

Construction Heuristic Description: This algorithm orders groups in decreasing width and iteratively adds groups to a row until the row is full. If the group cannot be fit in the current row, the algorithm moves on to the next row, until the last row has been used. 

Initialize: 
List of I groups ordered in decreasing group width. The max width for each row  is W =15.
Start with no utilization of rows, J=0.  Set remaining capacity or row j to RCj  = 15, where jJ. Starting OV(Total number of surplus seats in the theater) is  225. 


In [1]:
notPlaced=[]
bins =[]
difference=[]
diffNum = 0;
count=0;
m=0;

def pack(values, maxValue):
    values = sorted(values, reverse=True)
    count =0;
    for item in values:
        # Try to fit item into a bin
        for bin in bins:
            if sum(bin) + item <= maxValue:
                bin.append(item)
                break
        if len(bins)<15:
            # item didn't fit into any bin, start a new bin
            bin = [item]
            bins.append(bin)
        else:
            notPlaced.append(item)
    print('Items not Placed',notPlaced)
    for bin in bins:
        count= count +1
        print('Row ',count, " contains the groups:" ,bin, "and the number of surplus seats are:", maxValue-sum(bin))
        difference.append(maxValue -sum(bin))
        global diffNum 
        diffNum = sum(difference)
    print('Total number of surplus seats in the theater :', diffNum)

    return bins
   

if __name__ == '__main__':
    import random

    def packNums(aList, maxValue):
        bins = pack(aList, maxValue)
        print ('Solution using', len(bins), 'rows:')
        emptySeats= []
        sum=0;
        for bin in bins:
            print(bin)
        print

    aList = [ random.randint(3, 10) for i in range(31) ]
    packNums(aList, 15)


Items not Placed [6, 6, 6, 5, 5, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3]
Row  1  contains the groups: [10, 5] and the number of surplus seats are: 0
Row  2  contains the groups: [10, 5] and the number of surplus seats are: 0
Row  3  contains the groups: [10, 5] and the number of surplus seats are: 0
Row  4  contains the groups: [10, 4] and the number of surplus seats are: 1
Row  5  contains the groups: [10, 4] and the number of surplus seats are: 1
Row  6  contains the groups: [10, 4] and the number of surplus seats are: 1
Row  7  contains the groups: [9, 6] and the number of surplus seats are: 0
Row  8  contains the groups: [9, 6] and the number of surplus seats are: 0
Row  9  contains the groups: [9, 6] and the number of surplus seats are: 0
Row  10  contains the groups: [8, 7] and the number of surplus seats are: 0
Row  11  contains the groups: [7, 7] and the number of surplus seats are: 1
Row  12  contains the groups: [7, 6] and the number of surplus seats are: 2
Row  13  contains the gro

## Simulated Annealing 

Initial trial solution: Start with the initial feasible solution from the construction heuristic.
Neighborhood structure: Group swap - remove group from a row, replace with an unseated group from array Z. 
Random selection of an immediate neighbor: select the first item in the next bin to remove, add from unselected groups until full. If the group cannot be fit in the current row, the algorithm moves on to the next row, until the last row has been used. 

Perform n = 8 iterations where m = 2, k = 4
T1 = 0.5Zc , Tk = 0.2Zk-1 


In [6]:
#simulated annealing
#4 functions, one that does the loop calls both functions
# another that packs
# another that removes
# another that does the probability and stats 

#append to the notPlaced but then sort them 

import math
import random
from copy import deepcopy

m=0;

values=[]

print('original OV, i.e # of surplus seats in the theater',diffNum)
newDiff = diffNum
newBins = deepcopy(bins);
global newBins

e= 2.718281828459045090795598298428
T=deepcopy(newDiff)*0.5
Tk=0.2*T

q=0
iterPass= 0;
firstEnter=0;

def check(differenceD):
    global m
    global q
    global newDiff
    global iterPass
    if differenceD <= newDiff: 
        iterPass = iterPass +1
        print('print iteration', iterPass)
        newDiff = differenceD
        m= m+1
        q=0
        print('better or same sol with #of surplus seats in the theater = ', newDiff)
        Tsched(m)

    else:
        ('print iteration', iterPass)
        if q==2: #if the trials is reached 2, reset to 0, increase iteration
            iterPass= iterPass+1
            q=0
            m=m+1
        else:
            global T
            global p
            global Tk
            global firstEnter
            n= random.random()
            if  firstEnter==0:
                p=e**((newDiff -differenceD)/T) 
                firstEnter=1;
            if  firstEnter> 0:
                T=T*0.2
                p=e**((newDiff -differenceD)/T) 
            if  differenceD > newDiff and n <=p:  
                newDiff = differenceD
                m=m+1
                q =q+1
                print('chose a worst solution, new diff :', newDiff)
                Tsched(m)
            else:
                print('reverted bin from',newBins[m])
                newBins[m]= bins[m]
                print('to:',newBins[m])
                print('denied a worst solution')
                m=m+1
                q =q+1
                Tsched(m)    
            

def removeVal(newBin):
    notPlaced.sort()
    notPlaced.append(newBin[0])
    newBin.pop(0)
    simPack(newBins,15)
    
def simPack(newBins,maxValue):
    print('list before packed:', notPlaced)
    count =0;
    for item in notPlaced:
        # Try to fit item into a bin
        for newBin in newBins:
            if sum(newBin) + item <= maxValue:
                newBin.append(item)
                notPlaced.remove(item)
                break
    differenceD=0;
    annealD=[];
        
    for newBin in newBins:
        print("row: ",newBin,"and # of surplus seats", maxValue-sum(newBin))
        annealD.append(maxValue -sum(newBin))
        differenceD = sum(annealD)
    print('Total number of surplus seats in the theater :', differenceD)
    print('iterpass', iterPass)
    
    if iterPass<9:
        check(differenceD)
        
    print("\n")
    return notPlaced
    return newBins


def Tsched(m):
    if m<16:
        print("Iteration :",m)
        removeVal(newBins[m])
    
Tsched(0)



original OV, i.e # of surplus seats in the theater 7
Iteration : 0
list before packed: [3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 6, 6, 6, 7, 7, 7, 10]
row:  [5, 3, 3, 4] and # of surplus seats 0
row:  [10, 5] and # of surplus seats 0
row:  [10, 4] and # of surplus seats 1
row:  [9, 6] and # of surplus seats 0
row:  [9, 6] and # of surplus seats 0
row:  [9, 6] and # of surplus seats 0
row:  [9, 4] and # of surplus seats 2
row:  [9, 4] and # of surplus seats 2
row:  [8, 7] and # of surplus seats 0
row:  [8, 7] and # of surplus seats 0
row:  [8, 7] and # of surplus seats 0
row:  [8, 7] and # of surplus seats 0
row:  [8, 7] and # of surplus seats 0
row:  [7, 4, 4] and # of surplus seats 0
row:  [7, 3, 3] and # of surplus seats 2
Total number of surplus seats in the theater : 7
iterpass 0
print iteration 1
better or same sol with #of surplus seats in the theater =  7
Iteration : 1
list before packed: [3, 4, 4, 4, 4, 5, 5, 6, 6, 6, 7, 7, 7, 10, 10]
row:  [5, 3, 3, 4] and # of surplus seats 0
row:  [5, 